<a href="https://colab.research.google.com/github/ProfAI/tf00/blob/master/1%20-%20Le%20Basi%20di%20Tensorflow/classe_di_matematica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chi passerà matematica ?
La classificazione è il problema in cui cerchiamo di approssimare una funzione il cui output è una classe, cioè un valore appartenente ad un set di valori finiti.
Per approcciarci alla classificazione, in questo notebook addestreremo un modello di machine learning per predire quali studenti passeranno la classe di matematica, quindi i possibili output del modello sono solamente 2: promosso o bocciato.

## Importiamo i Moduli
Per scaricare ed operare sul dataset utilizzeremo il modulo Pandas.

In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

## Creiamo il Dataset

Utilizziamo Pandas per caricare il file csv contenente le informazioni sugli studenti direttamente dentro un DataFrame, un'oggetto che Pandas utilizza per rappresentare dati in forma tabulare.

In [0]:
csv_url = "https://raw.githubusercontent.com/ProfAI/tf00/master/datasets/math_class/math_class.csv"
df = pd.read_csv(csv_url) # carichiamo il csv in un DataFrame
df.head() # stampiamo le prime 5 righe

,student_id,studytime,failures,absences,promoted
0,0,2,0,6,0
1,1,2,0,4,0
2,2,2,3,10,1
3,3,3,0,2,1
4,4,2,0,4,1


Ogni riga rappresenta uno studente e ogni colonna una determinata informazione. Le informazioni che abbiamo a disposizione sono le seguenti:
* **student_id**: codice identificativo dello studente.
* **studytime**: ore di studio al giorno.
* **failures**: numero di volte che lo studene è stato rimandato in matematica in passato.
* **absences**: ore di assenze nella classe di matematica per il quadrimestre corrente.
* **promoted**: variabile booleana (True=1,False=0) che indica se lo studente è stato promosso.

**promoted** è l'informazione che dobbiamo predire, quindi il target dentro modello, mentre utilizzeremo le altre colonne come features, ad eccezione di **student_id**, la quale non contiene alcuna informazione utile ai fini della classificazione.
<br>
Otteniamo i corrispondenti array numpy, per features e target, dal DataFrame.

In [0]:
X = df.drop(["promoted", "student_id"], axis=1).values
y = df[["promoted"]].values

## Creiamo il Modello
Addestriamo il modello, per eseguire la classificazione dobbiamo definire una *funzione di attivazione*, che per problemi di classificazione binaria (cioè tra due classi) è la sigmoide.

In [0]:
model = keras.models.Sequential([
    keras.layers.Dense(1, input_shape=[3], activation="sigmoid")
])

Utilizziamo la **Binary Crossentropy (o Log Loss)** come funzione di costo, la quale tiene conto della probabilità di quanto le predizione del modello siano corrette, e aggiungiamo **l'Accuracy** come metrica, che indica semplicemente la percentuale di classficazioni che il modello ha eseguito correttamente.

In [0]:
model.compile(optimizer="sgd", loss="binary_crossentropy", metrics=["accuracy"])

Avviamo l'addestramento per 100 epoche.

In [0]:
model.fit(X, y, epochs=100)

Epoch 1/100
13/13 [==============================] - 0s 1ms/step - loss: 1.2986 - accuracy: 0.4025
Epoch 2/100
13/13 [==============================] - 0s 1ms/step - loss: 0.7572 - accuracy: 0.6152
Epoch 3/100
13/13 [==============================] - 0s 1ms/step - loss: 0.7368 - accuracy: 0.6734
Epoch 4/100
13/13 [==============================] - 0s 1ms/step - loss: 0.7343 - accuracy: 0.6709
Epoch 5/100
13/13 [==============================] - 0s 1ms/step - loss: 0.7278 - accuracy: 0.6759
Epoch 6/100
13/13 [==============================] - 0s 1ms/step - loss: 0.7233 - accuracy: 0.6633
Epoch 7/100
13/13 [==============================] - 0s 1ms/step - loss: 0.7196 - accuracy: 0.6633
Epoch 8/100
13/13 [==============================] - 0s 1ms/step - loss: 0.7172 - accuracy: 0.6734
Epoch 9/100
13/13 [==============================] - 0s 1ms/step - loss: 0.7127 - accuracy: 0.6684
Epoch 10/100
13/13 [==============================] - 0s 1ms/step - loss: 0.7096 - accuracy: 0.6759
Epoch 11/

Il valore della Binary Crossentropy (o Log Loss) all'ultima epoca è 0.5986, che è discreto ma non eccellente, risultato rispecchiato anche dall'accuracy: 0.719, questo vuol dire che il modello ha classificato correttamente il 72% degli esempi del set di addestramento.

## I miei figli verranno bocciati ?
Ora che abbiamo un modello discreto addestrato, utilizziamolo per cercare di sapere preventivamente se i miei 3 figli verranno bocciato, questi sono i loro dati:
* **Alex**: non studia per niente, è già stato rimandato in matematica 3 volte e questo trimestre ha già fatto 10 assenze.
* **Albert**: studia 6 ore al giorno, non è mai stato rimandato ed ha fatto solo 2 assenze.
* **Galileo**: studia 2 ore al giorno, in passato è stato rimandato una volta e questo trimestre ha fatto 5 assenze.

In [0]:
X_test = [[0, 3, 10], [6, 0, 2], [2, 1, 5]]
model.predict(X_test)

array([[0.18776762],
       [0.9094189 ],
       [0.58225733]], dtype=float32)

In caso di classificazione, utilizzando la sigmoide come funzione di attivazione, il metodo *predict* ritorna la probabilità di apperteneza alla classe positiva, in questo caso la probabilità che l'utente venga promosso in matematica. I nostri risultati sono i seguenti:
* **Alex**: 18.7% di probabilità di venire promosso.
* **Albert**: 91% di probabilità di venire promosso.
* **Galileo**: 58.2% di probabilità di venire promosso.

Se piuttosto che la probabilità vogliamo ottenere direttamente la classe di appartenenza (1=classe positiva=promosso, 0=classe negativa=bocciato), possiamo usare il metodo *predict_classes*.

In [0]:
model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([[0],
       [1],
       [1]], dtype=int32)

# Qual è la "Formula della Promozione" ?
Anche in questo caso possiamo estrarre dal modello pesi e bias e ricostruire la relazione tra features e target.

In [0]:
model.weights

[<tf.Variable 'dense/kernel:0' shape=(3, 1) dtype=float32, numpy=
 array([[ 0.3558504 ],
        [-0.4992251 ],
        [-0.01729545]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.2060485], dtype=float32)>]

$$y=0.36x_1-\frac{x_2}{2}-0.018x_3+\frac{1}{5}$$